In [15]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq groq
import os
from groq import Groq
import getpass
client = Groq()

In [22]:
EXPERT_CONFIGS = {
    "technical": {
        "system_prompt": "You are a Senior Software Engineer. Provide rigorous, code-focused, and precise solutions. Always check for syntax and logic errors.",
        "model": "llama-3.1-8b-instant"
    },
    "billing": {
        "system_prompt": "You are a Billing Specialist. You are empathetic, focused on financial accuracy, and strictly adhere to company refund and subscription policies.",
        "model": "llama-3.1-8b-instant"
    },
    "general": {
        "system_prompt": "You are a helpful General Support Assistant. Use a friendly tone and provide concise answers for non-technical and non-billing queries.",
        "model": "llama-3.1-8b-instant"
    }
}

In [23]:
def route_prompt(user_input):
    routing_instructions = (
        "Classify this text into one of these categories: [technical, billing, general]. "
        "Return ONLY the word of the category, nothing else."
    )

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": routing_instructions},
            {"role": "user", "content": user_input}
        ],
        model="llama-3.1-8b-instant",
        temperature=0
    )

    category = response.choices[0].message.content.strip().lower()
    return category

def process_request(user_input):

    category = route_prompt(user_input)
    print(f"--- [Router Log]: Routing to '{category}' expert ---")

    if category not in EXPERT_CONFIGS:
        category = "general"

    expert = EXPERT_CONFIGS[category]

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": expert["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        model=expert["model"],
        temperature=0.7
    )

    return response.choices[0].message.content

In [24]:
# Test Case 1: Technical
print("\nInput: 'My python script is throwing an IndexError on line 5.'")
print("Response:", process_request("My python script is throwing an IndexError on line 5."))

print("-" * 30)

# Test Case 2: Billing
print("\nInput: 'I was charged twice for my subscription this month.'")
print("Response:", process_request("I was charged twice for my subscription this month."))

print("-" * 30)

# Test Case 3: General
print("\nInput: 'What time do you close on Fridays?'")
print("Response:", process_request("What time do you close on Fridays?"))


Input: 'My python script is throwing an IndexError on line 5.'
--- [Router Log]: Routing to 'technical' expert ---
Response: To help you resolve the issue, I'll need more information about your code. Please provide the following:

1. The exact error message, including the line number.
2. The relevant code snippet that's causing the error.
3. The expected input or data that you're working with.

However, I can give you a general outline on how to debug the issue:

1. **Check the list index**: Make sure you're not trying to access an index that's out of range for the list. In Python, indices start at 0.
2. **Verify list length**: Ensure the list has at least the number of elements you're trying to access.
3. **Check for empty lists**: If you're working with lists, ensure they're not empty before trying to access their elements.

Here's a simple example to demonstrate how to debug an IndexError:

```python
# Define a list with 3 elements
my_list = [1, 2, 3]

# Try to access the 4th eleme